# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize
from collections import Counter

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


In [6]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [7]:
import json
from pandas.io.json import json_normalize

from collections import Counter

In [8]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

- The code below accesses the nested data in the colum'njtheme_namecode' and uses a dictionary to populates it's empty rows with the map method.

- I Learned how to access and  make sense of nested data by researching and using json_normalize

- I created a series (pre_dict) which could be  turned into a dictionary with less difficulty than the original dataframe

- I decided to create a dictionary with keys that were the data from the 'name' column as opposed to using data from the  'code' column. This way the dictionary didn't have keys with empty values

- After making the original dictionary I reveresed the keys and values in the dictionary and then mapped the 'name' coLumn with 'rev_dict' so that there were no empty values


In [9]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
with open('data/world_bank_projects.json') as jsonfile:
        f = json.load(jsonfile)
theme_df = json_normalize(f, record_path ='mjtheme_namecode', meta = ['id'])
pre_dict = theme_df.set_index('name')['code']

coded_dict = pre_dict.to_dict()
rev_dict = {v:k for k,v in coded_dict.items()}
theme_df['name']=theme_df.code.map(rev_dict)
print(theme_df)
        


     code                                          name       id
0       8                             Human development  P129828
1      11  Environment and natural resources management  P129828
2       1                           Economic management  P144674
3       6         Social protection and risk management  P144674
4       5                         Trade and integration  P145310
5       2                      Public sector governance  P145310
6      11  Environment and natural resources management  P145310
7       6         Social protection and risk management  P145310
8       7                   Social dev/gender/inclusion  P144665
9       7                   Social dev/gender/inclusion  P144665
10      5                         Trade and integration  P144933
11      4      Financial and private sector development  P144933
12      6         Social protection and risk management  P146161
13      6         Social protection and risk management  P146161
14      2                

- The following two Cells of code use practically the same code as eachother.  The main difference is the project_theme value_count used the data that was already extracted in the above dataframe, while the second cell of code used read_json to get the data. I used read_json because the data necessary for that exercise was not nested.

- Because I removed the rows with blank project themes above I did not have to deal with NaN values

- There were multiple approaches I could have taken listing the most common countries and project themes. One that came to mind was the Counter tool from the collections module. I used the value_counts method because i felt it was cleaner and required less code
    
- When value counts produced series I turned them into dataframes so that I could more easily create labels and names


In [10]:

project_themes = (theme_df.name.value_counts().nlargest(n=10))
project_themes = project_themes.to_frame(name = 'Number per Project Theme')
project_themes.index.name = 'TEN MAJOR PROJECT THEMES'
print(project_themes)

                                              Number per Project Theme
TEN MAJOR PROJECT THEMES                                              
Environment and natural resources management                       250
Rural development                                                  216
Human development                                                  210
Public sector governance                                           199
Social protection and risk management                              168
Financial and private sector development                           146
Social dev/gender/inclusion                                        130
Trade and integration                                               77
Urban development                                                   50
Economic management                                                 38


In [11]:
wb_data = pd.read_json('data\world_bank_projects.json')
ten_countries = (wb_data.countryname.value_counts().nlargest(n=10))
new_labels = [ 'Number of World Bank Projects']
ten_countries = ten_countries.to_frame()
ten_countries.columns = new_labels
ten_countries.index.name = "COUNTRY NAMES"
print(ten_countries)


                                 Number of World Bank Projects
COUNTRY NAMES                                                 
Republic of Indonesia                                       19
People's Republic of China                                  19
Socialist Republic of Vietnam                               17
Republic of India                                           16
Republic of Yemen                                           13
People's Republic of Bangladesh                             12
Nepal                                                       12
Kingdom of Morocco                                          12
Republic of Mozambique                                      11
Africa                                                      11
